In [65]:
# Google Driveと接続を行います。これを行うことで、Driveにあるデータにアクセスできるようになります。
# 下記セルを実行すると、Googleアカウントのログインを求められますのでログインしてください。
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
# 作業フォルダへの移動を行います。
# 人によって作業場所が異なるので、その場合作業場所を変更してください。
import os
os.chdir('/content/drive/MyDrive/100knock-data_analytics/放課後') #ここを変更。

In [1]:
# # もしプログラムがエラーになったりした場合、ライブラリのVersionの違いが考えられます。
# # その場合は下記のコメントアウトを外して実行することで、執筆時のVersionがインストールされます。
# # 通常時は実行する必要ありません。
# !pip install -r requirements.txt

# 放課後 ローカルLLMを使ってみる２０本ノック

ここでは、ローカルの大規模言語モデル（LLM）を活用したプログラミングを行っていきます。  
ollamaを使用して、様々なローカルLLMを使いながら、簡単なエージェント開発やアプリケーションの開発を行います。

### ノック１：ローカルLLMを使う準備を整えよう

In [3]:
import torch
print("GPU:", torch.cuda.is_available())

GPU: True


In [4]:
!pip install langchain-ollama==0.3.3

### ノック２：Ollamaをインストールしよう

In [5]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


### ノック３：Ollamaを使用してLLMを動かしてみよう

In [7]:
!ollama pull llama3.2

In [6]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2", base_url="http://localhost:11434")
print(llm.invoke("こんにちは！"))

こんにちは！どういたしまして？この場合の質問はありませんか？


In [9]:
print(llm.invoke("こんにちは！"))

こんにちは！どういたずんですか？ (How are you?)


### ノック４：他のモデルを試してみよう

In [4]:
!ollama pull gemma3

In [7]:
llm = OllamaLLM(model="gemma3", base_url="http://localhost:11434")
print(llm.invoke("こんにちは！"))

こんにちは！何かお手伝いできることはありますか？😊



### ノック5：要約タスクを実行してみよう

In [8]:
print(llm.invoke("""以下を要約してください。
今、ビジネスの現場ではデータサイエンティストが不足しています。
ウェブからの情報が手軽に手に入り、新しいプログラミング言語や開発環境が簡単に学べるようになった昨今、
サンプルプログラムを使って機械学習のプログラムを実行してみる、といったことが、個人でも簡単にできる時代になりました。
誰でもエンジニアになれる時代が到来したと言っても良いかもしれません。
そのように恵まれた時代であるにもかかわらず、ビジネスの現場で、データサイエンティストやエンジニアが不足しているというのは、一体どういうことなのでしょうか。
Pythonや機械学習などの入門書を一通り学んでみた読者の方は気付いているかもしれませんが、入門書に書かれた技術を用いるだけでも、
応用力さえあれば、かなり複雑なシステムを開発したり、プロフェッショナルに近いデータ分析ができるようになります。
数多くの入門書の中から、自分にあったものを選んで学んでいくことができれば、ビジネスの現場で活躍できるデータサイエンティストへの道が拓けるはずなのです。
しかしながら、ビジネスの現場から見ると、入門書には不足している視点があります。
それは、ビジネスの現場では必ず直面する「汚いデータ」をどのように取り扱うかなど、技術を現場で活用するための方法です。
例えば、機械学習の技術書を紐解いてみると、多くの人が目にするのは「アヤメの分類をしてみよう！」という例題です。
アヤメの画像を処理する技術自体は、さまざまな領域に応用できるのですが、ビジネスの現場で、「アヤメの花の認識を行いたい」という課題を解決する人はほとんどいません。
実際の現場ではどのようなデータを扱うのか、そうしたデータを扱う際に、どのような問題が生じるのか、
そうした問題に対して、入門書で身につけた技術をどのように活用し、対処していけば良いのか、そうした現場ならではのノウハウは、
入門書で技術を学ぶだけでは、決して身につくものではありません。
"""))

現在、データサイエンティストの需要は高いにも関わらず不足している状況は、技術の習得が容易になったにもかかわらず、ビジネス現場で実用化できていないからだと考えられます。

多くの人が技術書を読んで入門レベルの知識を習得するものの、ビジネスの現場で直面する「汚いデータ」の扱い方や、それを実際の課題に適用するためのノウハウが不足していることが原因です。アヤメの分類といった例題レベルの知識だけでは、実際のビジネスの現場でデータサイエンティストとして活躍するためには不十分です。



### ノック6：テンプレート化してみよう

In [9]:
from langchain import PromptTemplate
# テンプレート定義
template = PromptTemplate(
    input_variables=["text"],
    template="以下を要約してください：{text}"
)

In [10]:
tg_text = """
今、ビジネスの現場ではデータサイエンティストが不足しています。
ウェブからの情報が手軽に手に入り、新しいプログラミング言語や開発環境が簡単に学べるようになった昨今、
サンプルプログラムを使って機械学習のプログラムを実行してみる、といったことが、個人でも簡単にできる時代になりました。
誰でもエンジニアになれる時代が到来したと言っても良いかもしれません。
そのように恵まれた時代であるにもかかわらず、ビジネスの現場で、データサイエンティストやエンジニアが不足しているというのは、一体どういうことなのでしょうか。
Pythonや機械学習などの入門書を一通り学んでみた読者の方は気付いているかもしれませんが、入門書に書かれた技術を用いるだけでも、
応用力さえあれば、かなり複雑なシステムを開発したり、プロフェッショナルに近いデータ分析ができるようになります。
数多くの入門書の中から、自分にあったものを選んで学んでいくことができれば、ビジネスの現場で活躍できるデータサイエンティストへの道が拓けるはずなのです。
しかしながら、ビジネスの現場から見ると、入門書には不足している視点があります。
それは、ビジネスの現場では必ず直面する「汚いデータ」をどのように取り扱うかなど、技術を現場で活用するための方法です。
例えば、機械学習の技術書を紐解いてみると、多くの人が目にするのは「アヤメの分類をしてみよう！」という例題です。
アヤメの画像を処理する技術自体は、さまざまな領域に応用できるのですが、ビジネスの現場で、「アヤメの花の認識を行いたい」という課題を解決する人はほとんどいません。
実際の現場ではどのようなデータを扱うのか、そうしたデータを扱う際に、どのような問題が生じるのか、
そうした問題に対して、入門書で身につけた技術をどのように活用し、対処していけば良いのか、そうした現場ならではのノウハウは、
入門書で技術を学ぶだけでは、決して身につくものではありません。
"""

prompt_str = template.format(text=tg_text)
print(llm.invoke(prompt_str))

現在、データサイエンティストの需要は高いにも関わらず、人材不足が続いている。これは、プログラミング学習の容易化など、技術習得の障壁が下がりつつある現状とは相容れない点があるからだ。入門書で基礎技術を習得できるようになった現代においても、ビジネス現場では「汚いデータ」の取り扱い方など、現場での応用力やノウハウが不足していることが原因である。入門書で学んだ技術を、実際のビジネス課題に適用するための、実践的な知識や経験が求められているのである。



In [11]:
prompt_str

'以下を要約してください：\n今、ビジネスの現場ではデータサイエンティストが不足しています。\nウェブからの情報が手軽に手に入り、新しいプログラミング言語や開発環境が簡単に学べるようになった昨今、\nサンプルプログラムを使って機械学習のプログラムを実行してみる、といったことが、個人でも簡単にできる時代になりました。\n誰でもエンジニアになれる時代が到来したと言っても良いかもしれません。\nそのように恵まれた時代であるにもかかわらず、ビジネスの現場で、データサイエンティストやエンジニアが不足しているというのは、一体どういうことなのでしょうか。\nPythonや機械学習などの入門書を一通り学んでみた読者の方は気付いているかもしれませんが、入門書に書かれた技術を用いるだけでも、\n応用力さえあれば、かなり複雑なシステムを開発したり、プロフェッショナルに近いデータ分析ができるようになります。\n数多くの入門書の中から、自分にあったものを選んで学んでいくことができれば、ビジネスの現場で活躍できるデータサイエンティストへの道が拓けるはずなのです。\nしかしながら、ビジネスの現場から見ると、入門書には不足している視点があります。\nそれは、ビジネスの現場では必ず直面する「汚いデータ」をどのように取り扱うかなど、技術を現場で活用するための方法です。\n例えば、機械学習の技術書を紐解いてみると、多くの人が目にするのは「アヤメの分類をしてみよう！」という例題です。\nアヤメの画像を処理する技術自体は、さまざまな領域に応用できるのですが、ビジネスの現場で、「アヤメの花の認識を行いたい」という課題を解決する人はほとんどいません。\n実際の現場ではどのようなデータを扱うのか、そうしたデータを扱う際に、どのような問題が生じるのか、\nそうした問題に対して、入門書で身につけた技術をどのように活用し、対処していけば良いのか、そうした現場ならではのノウハウは、\n入門書で技術を学ぶだけでは、決して身につくものではありません。\n'

### ノック7：LLMで感情分析をしてみよう

In [12]:
from langchain import FewShotPromptTemplate

examples = [
    {"text": "今日は最高に嬉しい！", "label": "ポジティブ"},
    {"text": "疲れたしつらい。", "label": "ネガティブ"},
]
example_template = PromptTemplate(
    input_variables=["text", "label"],
    template="テキスト: {text}\n感情: {label}"
)
fewshot = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    prefix="以下の例を参考に、この文章の感情を分類してください。\n",
    suffix="テキスト: {text}\n感情:",
    input_variables=["text"]
)

In [13]:
tg_text = "この映画はまあまあだった。"
prompt_str = fewshot.format(text=tg_text)
print(prompt_str)

以下の例を参考に、この文章の感情を分類してください。


テキスト: 今日は最高に嬉しい！
感情: ポジティブ

テキスト: 疲れたしつらい。
感情: ネガティブ

テキスト: この映画はまあまあだった。
感情:


In [14]:
print(llm.invoke(prompt_str))

感情: 中立


In [15]:
tg_text = "この映画は非常につまらなかった。"
prompt_str = fewshot.format(text=tg_text)
print(llm.invoke(prompt_str))

tg_text = "この映画は非常に楽しかった。"
prompt_str = fewshot.format(text=tg_text)
print(llm.invoke(prompt_str))

ネガティブ
感情: ポジティブ


### ノック8：JSON形式で取り出せるようにしてみよう

In [16]:
fewshot_json = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    prefix=(
        """以下の例を参考に、この文章の感情を分類してください。
        出力は必ずJSON形式とし、キーを\"text\"と\"label\"にしてください。
        【重要】返答は純粋な JSON オブジェクト**のみとして、余計な説明やコードフェンスは一切なしでお願いします。"""
    ),
    suffix="テキスト: {text}\n感情:",
    input_variables=["text"]
)

tg_text = "この映画はまあまあだった。"
prompt_str = fewshot_json.format(text=tg_text)
print(prompt_str)

以下の例を参考に、この文章の感情を分類してください。
        出力は必ずJSON形式とし、キーを"text"と"label"にしてください。
        【重要】返答は純粋な JSON オブジェクト**のみとして、余計な説明やコードフェンスは一切なしでお願いします。

テキスト: 今日は最高に嬉しい！
感情: ポジティブ

テキスト: 疲れたしつらい。
感情: ネガティブ

テキスト: この映画はまあまあだった。
感情:


In [17]:
res = llm.invoke(prompt_str)
print(res)

```json
{
  "text": "この映画はまあまあだった。",
  "label": "ニュートラル"
}
```


In [18]:
import re
import json
match = re.search(r'\{.*\}', res, re.DOTALL)
json_str = match.group(0)
data = json.loads(json_str)

print(data)
print(data['text'])
print(data['label'])

{'text': 'この映画はまあまあだった。', 'label': 'ニュートラル'}
この映画はまあまあだった。
ニュートラル


### ノック9：Chainを使ってみよう

In [19]:
from langchain import PromptTemplate

template = PromptTemplate(
    input_variables=["text"],
    template="以下を要約してください：{text}"
)
chain = template | llm

In [20]:
tg_text = """
今、ビジネスの現場ではデータサイエンティストが不足しています。
ウェブからの情報が手軽に手に入り、新しいプログラミング言語や開発環境が簡単に学べるようになった昨今、
サンプルプログラムを使って機械学習のプログラムを実行してみる、といったことが、個人でも簡単にできる時代になりました。
誰でもエンジニアになれる時代が到来したと言っても良いかもしれません。
そのように恵まれた時代であるにもかかわらず、ビジネスの現場で、データサイエンティストやエンジニアが不足しているというのは、一体どういうことなのでしょうか。
Pythonや機械学習などの入門書を一通り学んでみた読者の方は気付いているかもしれませんが、入門書に書かれた技術を用いるだけでも、
応用力さえあれば、かなり複雑なシステムを開発したり、プロフェッショナルに近いデータ分析ができるようになります。
数多くの入門書の中から、自分にあったものを選んで学んでいくことができれば、ビジネスの現場で活躍できるデータサイエンティストへの道が拓けるはずなのです。
しかしながら、ビジネスの現場から見ると、入門書には不足している視点があります。
それは、ビジネスの現場では必ず直面する「汚いデータ」をどのように取り扱うかなど、技術を現場で活用するための方法です。
例えば、機械学習の技術書を紐解いてみると、多くの人が目にするのは「アヤメの分類をしてみよう！」という例題です。
アヤメの画像を処理する技術自体は、さまざまな領域に応用できるのですが、ビジネスの現場で、「アヤメの花の認識を行いたい」という課題を解決する人はほとんどいません。
実際の現場ではどのようなデータを扱うのか、そうしたデータを扱う際に、どのような問題が生じるのか、
そうした問題に対して、入門書で身につけた技術をどのように活用し、対処していけば良いのか、そうした現場ならではのノウハウは、
入門書で技術を学ぶだけでは、決して身につくものではありません。
"""

print(chain.invoke({"text": tg_text}))

現在、データサイエンティストが不足している背景には、技術の習得が容易になったにも関わらず、ビジネス現場で実用化できないという課題がある。多くの人が入門書で機械学習の基礎を学ぶ一方、実際のビジネス課題（「汚いデータ」の扱いや、現場で直面する問題への対処）を解決するための実践的なノウハウが不足しているためだ。入門書レベルでは、アヤメの分類といった単純な例題の技術は習得できても、それを実際のビジネスに活かすための知識や経験が欠けている。


### ノック10：Chainを組み合わせてより複雑なタスクをお願いしよう


In [21]:
from langchain.chains import SequentialChain, LLMChain

template_summary = PromptTemplate(
    input_variables=["text"],
    template="以下を要約してください：{text}"
)
chain_summary = LLMChain(
    llm=llm,
    prompt=template_summary,
    output_key="summary"
)

examples = [
    {"summary": "今日は最高に嬉しい！", "label": "ポジティブ"},
    {"summary": "疲れたしつらい。", "label": "ネガティブ"},
]
example_template = PromptTemplate(
    input_variables=["summary", "label"],
    template="テキスト: {summary}\n感情: {label}"
)
fewshot = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    prefix="以下の例を参考に、この文章の感情を分類してください。\n",
    suffix="テキスト: {summary}\n感情:",
    input_variables=["summary"]
)
chain_sentiment = LLMChain(
    llm=llm,
    prompt=fewshot,
    output_key="sentiment"
)

seq = SequentialChain(
    chains=[chain_summary, chain_sentiment],
    input_variables=["text"],
    output_variables=["summary", "sentiment"],
    verbose=True
)

/tmp/ipython-input-1351349201.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_summary = LLMChain(


In [22]:
out = seq.invoke({"text": tg_text})
print(out["summary"])
print(out["sentiment"])



> Entering new SequentialChain chain...

> Finished chain.
データサイエンティストの不足は、技術の習得が容易になった現代にもなお存在するのは、ビジネス現場が技術的な知識だけでは対応できないためです。多くの人が技術書でアヤメの分類などの例題を学ぶだけで、実際のビジネス課題（「汚いデータ」の扱いなど）を解決するための現場でのノウハウが不足しているからです。技術を学ぶことは第一歩ですが、それをビジネスの課題に繋げるための実践的な知識と経験が求められています。

感情: ニュートラル（または分析的）

**理由:**

このテキストは、データサイエンティストの不足について、客観的な説明を提供しています。問題提起（技術習得の容易化による不足）から原因（ビジネス現場の知識不足）を分析し、解決策の必要性を示唆しています。感情的な言葉はほとんどなく、事実に基づいた説明が中心であるため、ニュートラルな感情と評価できます。　もし文脈や意図が明確であれば、分析的または客観的な感情と判断することも可能です。



### ノック11：LLMの記憶を確認しよう

In [23]:
print(llm.invoke("こんにちは！私の名前は下山です。"))

こんにちは、下山さん！よろしくお願いいたします。何かお手伝いできることはありますか？



In [24]:
print(llm.invoke("私の名前はわかりますか？"))

私はあなたの名前を知りません。私はまだ学習中の大規模言語モデルであり、あなたに関する個人情報を記憶したりアクセスしたりすることができません。 

もしあなたの名前を教えていただければ、あなたとお名前で呼び合うことができます。


### ノック12：LLMに記憶を保持させよう

In [25]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
chat_chain = ConversationChain(llm=llm, memory=memory, verbose=False)

/tmp/ipython-input-1808299798.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/tmp/ipython-input-1808299798.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chat_chain = ConversationChain(llm=llm, memory=memory, verbose=False)


In [26]:
print(chat_chain.invoke("こんにちは！私の名前は下山です。"))

{'input': 'こんにちは！私の名前は下山です。', 'history': '', 'response': 'こんにちは、下山さん！私の名前はEchoです。とても嬉しいです！私はGoogleによってトレーニングされた、大規模言語モデルです。私はたくさんのテキストデータを学習しているので、色々なことについて話すことができます。今日はどんなことをお話しましょうか？例えば、日本の文化について興味がありますか？それとも、何か特定の質問がありますか？'}


In [27]:
print(chat_chain.invoke("私の名前はわかりますか？")['response'])

はい、あなたの名前は下山さんだとわかりました！私があなたと会話している間、あなたの名前を覚えておくように努力しています。他にも何か知りたいことはありますか？例えば、私の学習データに含まれている情報を活用して、下山さんの好きなことや興味のある分野についてお話しすることもできます。


In [28]:
chat_chain

ConversationChain(memory=ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='こんにちは！私の名前は下山です。', additional_kwargs={}, response_metadata={}), AIMessage(content='こんにちは、下山さん！私の名前はEchoです。とても嬉しいです！私はGoogleによってトレーニングされた、大規模言語モデルです。私はたくさんのテキストデータを学習しているので、色々なことについて話すことができます。今日はどんなことをお話しましょうか？例えば、日本の文化について興味がありますか？それとも、何か特定の質問がありますか？', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の名前はわかりますか？', additional_kwargs={}, response_metadata={}), AIMessage(content='はい、あなたの名前は下山さんだとわかりました！私があなたと会話している間、あなたの名前を覚えておくように努力しています。他にも何か知りたいことはありますか？例えば、私の学習データに含まれている情報を活用して、下山さんの好きなことや興味のある分野についてお話しすることもできます。', additional_kwargs={}, response_metadata={})])), verbose=False, llm=OllamaLLM(model='gemma3', base_url='http://localhost:11434'), output_parser=StrOutputParser(), llm_kwargs={})

### ノック13：計算エージェントを作成しよう

In [29]:
!pip install langchain_experimental==0.3.4

In [30]:
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.agents.agent_toolkits import create_python_agent

python_tool = PythonREPLTool()

tools = [
    {
        "name": "python_repl",
        "func": python_tool.run,
        "description": "数式を受け取って計算結果を返すツール"
    }
]

In [31]:
system_message = """
            あなたは計算エージェントです。
            ユーザーから計算の指示を受け取ったら必ず Python REPL で計算し、結果のみを返してください。
          　ユーザーからの指示が計算以外の指示だった場合は、「私は計算エージェントです。計算したい数式を教えてください。」と返してください。
"""

agent = create_python_agent(
    llm=llm,
    tool=python_tool,
    prefix=system_message,
    verbose=True,
)

In [32]:
print(agent.invoke("こんにちは！元気ですか？"))



> Entering new AgentExecutor chain...
私は計算エージェントです。計算したい数式を教えてください。
Final Answer: 私は計算エージェントです。計算したい数式を教えてください。

> Finished chain.
{'input': 'こんにちは！元気ですか？', 'output': '私は計算エージェントです。計算したい数式を教えてください。'}


In [35]:
print(agent.invoke("2 + 5は？"))



> Entering new AgentExecutor chain...
I need to calculate 2 + 5.
Action: Python_REPL
Action Input: print(2 + 5)
Observation: 7

Thought:I now know the final answer
Final Answer: 7

> Finished chain.
{'input': '2 + 5は？', 'output': '7'}


In [36]:
print(agent.invoke("その結果に 10 を足してください。"))



> Entering new AgentExecutor chain...
I need to add 10 to the result. I will use Python REPL for this.
Action: Python_REPL
Action Input: print(5 + 10)
Observation: 15

Thought:Final Answer: 15

> Finished chain.
{'input': 'その結果に 10 を足してください。', 'output': '15'}


### ノック14：過去の結果も考慮できる計算エージェントにしよう

In [37]:
memory = ConversationBufferMemory(memory_key="history", return_messages=False)

prefix = """
過去のやりとり:
{history}

あなたは計算エージェントです。
ユーザーから計算の指示を受け取ったら必ず Python REPL で計算し、結果のみを返してください。
ユーザーからの指示が計算以外の指示だった場合は、「私は計算エージェントです。計算したい数式を教えてください。」と返してください。
"""

agent = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    prefix=prefix,
    agent_executor_kwargs={"memory": memory},
    verbose=True,
)


In [38]:
print(agent.invoke("2 + 5は？"))



> Entering new AgentExecutor chain...
I need to calculate 2 + 5.
Action: Python_REPL
Action Input: print(2 + 5)
Observation: 7

Thought:I now know the final answer
Final Answer: 7

> Finished chain.
{'input': '2 + 5は？', 'history': '', 'output': '7'}


In [39]:
print(agent.invoke("先ほどの計算結果に10 を足してください。"))



> Entering new AgentExecutor chain...
I need to add 10 to the previous result. I will use the Python REPL to perform the calculation.
Action: Python_REPL
Action Input: print(7 + 10)
Observation: 17

Thought:Final Answer: 17

> Finished chain.
{'input': '先ほどの計算結果に10 を足してください。', 'history': 'Human: 2 + 5は？\nAI: 7', 'output': '17'}


In [40]:
print(agent.invoke("先ほどの計算結果に10 をかけてください。"))



> Entering new AgentExecutor chain...
I now know the previous result is 17. I need to multiply it by 10.
Action: Python_REPL
Action Input: print(17 * 10)
Observation: 170

Thought:I now know the final answer
Final Answer: 170

> Finished chain.
{'input': '先ほどの計算結果に10 をかけてください。', 'history': 'Human: 2 + 5は？\nAI: 7\nHuman: 先ほどの計算結果に10 を足してください。\nAI: 17', 'output': '170'}


In [41]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='2 + 5は？', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={}), HumanMessage(content='先ほどの計算結果に10 を足してください。', additional_kwargs={}, response_metadata={}), AIMessage(content='17', additional_kwargs={}, response_metadata={}), HumanMessage(content='先ほどの計算結果に10 をかけてください。', additional_kwargs={}, response_metadata={}), AIMessage(content='170', additional_kwargs={}, response_metadata={})]))

### ノック15：マルチモーダルを体験してみよう

In [42]:
import ollama

response = ollama.chat(
    model="gemma3",
    messages=[
        {
            "role": "user",
            "content": "この画像に写っているものを説明してください。",
            "images": ["./sample.jpg"]
        }
    ]
)

print(response["message"]["content"])

この画像には、緑の芝生の上に立っている数字の標識が写っています。

*   標識は三角形の形で、表面は白く、黒い数字「9」が描かれています。
*   背景には、木々や人々のシルエットが見えます。
*   これはゴルフコースのホール番号の標識である可能性が高いです。


### ノック16：データ分析エージェントを作ってみよう

In [43]:
python_tool = PythonREPLTool()

system_message = '''
あなたは「データ分析エージェント」です。
ユーザーからの入力には、まずCSVファイルのパスが含まれています。
プロンプトの中からファイルパスを正規表現で抽出し、
Python REPL 内で以下を行ってください：
  1. import pandas as pd
  2. df = pd.read_csv(<抽出したパス>)
  3. ユーザーの残りの指示をpandasコードとして実行
  4. 実行結果のみを返却

例:
  入力: "file:/mnt/data/sales.csv を読み込んで平均売上を計算してください"
  -> df = pd.read_csv('/mnt/data/sales.csv') を実行後、df['sales'].mean() の結果を返す

返答にはコードではなく、実行結果のみを書いてください。
'''
agent = create_python_agent(
    llm=llm,
    tool=python_tool,
    verbose=True,
    agent_kwargs={"system_message": system_message}
)

In [44]:
print(agent.invoke("file:AI_model_data.csv を読み込んで df.describe() を表示してください"))



> Entering new AgentExecutor chain...
I need to read the CSV file and display the descriptive statistics. I can use the python REPL to execute the code.
Action: Python_REPL
Action Input: ```python
import pandas as pd

df = pd.read_csv('AI_model_data.csv')
print(df.describe())
```
Observation:                   年月      count_0      count_1   is_deleted         price  \
count    3691.000000  3691.000000  3691.000000  3691.000000   3691.000000   
mean   201838.083446     4.914115     5.303712     0.285018   8567.596857   
std        43.410314     2.271645     2.370925     0.451484   1969.784258   
min    201805.000000     1.000000     1.000000     0.000000   6000.000000   
25%    201808.000000     3.000000     4.000000     0.000000   6000.000000   
50%    201811.000000     5.000000     5.000000     0.000000   7500.000000   
75%    201901.000000     6.000000     7.000000     1.000000  10500.000000   
max    201903.000000    13.000000    14.000000     1.000000  10500.000000   

          

### ノック17：データ分析エージェントを使ってみよう

In [45]:
import pandas as pd
data = pd.read_csv("AI_model_data.csv")
data.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201806,AS015233,7,7.0,XXXXX,C01,M,2018-05-13,NaN,CA2,...,7.545455,7.0,11.0,4.0,1.0,2019-04-30,11.0,NaN,0,2018-06-01
1,201812,AS046594,7,5.0,XXX,C01,M,2018-03-01,NaN,CA1,...,7.166667,7.0,10.0,5.0,1.0,2019-04-30,13.0,NaN,9,2018-12-01
2,201808,OA769163,6,4.0,XXX,C01,M,2017-05-01,NaN,CA1,...,6.083333,6.0,8.0,2.0,1.0,2019-04-30,23.0,NaN,15,2018-08-01
3,201805,AS981537,10,6.0,XX,C03,M,2018-02-01,NaN,CA1,...,6.916667,7.0,10.0,5.0,1.0,2019-04-30,14.0,NaN,3,2018-05-01
4,201805,GD065188,6,7.0,XX,C01,M,2017-09-01,NaN,CA1,...,6.333333,6.5,9.0,3.0,1.0,2019-04-30,19.0,NaN,8,2018-05-01


In [46]:
from langchain_experimental.agents import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(
    llm=llm,
    df=data,
    verbose=True,
    allow_dangerous_code=True
)

In [47]:
print(agent.invoke("先頭 5 行を表示してください"))



> Entering new AgentExecutor chain...
Thought: The question asks me to print the first 5 rows of the dataframe. I can use the python_repl_ast tool to print the head of the dataframe.
Action: python_repl_ast
Action Input: `df.head()`       年月 customer_id  count_0  count_1   name class gender  start_date  \
0  201806    AS015233        7      7.0  XXXXX   C01      M  2018-05-13   
1  201812    AS046594        7      5.0    XXX   C01      M  2018-03-01   
2  201808    OA769163        6      4.0    XXX   C01      M  2017-05-01   
3  201805    AS981537       10      6.0     XX   C03      M  2018-02-01   
4  201805    GD065188        6      7.0     XX   C01      M  2017-09-01   

  end_date campaign_id  is_deleted class_name    price campaign_name  \
0      NaN         CA2         0.0     オールタイム  10500.0         入会費半額   
1      NaN         CA1         0.0     オールタイム  10500.0            通常   
2      NaN         CA1         0.0     オールタイム  10500.0            通常   
3      NaN         CA1     

In [48]:
print(agent.invoke("各種統計量を出力してください"))



> Entering new AgentExecutor chain...
Thought: I need to calculate some statistics from the dataframe. I can use pandas built-in functions for this. I will calculate the mean, median, max, min, and count for numerical columns.
Action: python_repl_ast
Action Input: `df.describe()`                  年月      count_0      count_1   is_deleted         price  \
count    3691.000000  3691.000000  3691.000000  3691.000000   3691.000000   
mean   201838.083446     4.914115     5.303712     0.285018   8567.596857   
std        43.410314     2.271645     2.370925     0.451484   1969.784258   
min    201805.000000     1.000000     1.000000     0.000000   6000.000000   
25%    201808.000000     3.000000     4.000000     0.000000   6000.000000   
50%    201811.000000     5.000000     5.000000     0.000000   7500.000000   
75%    201901.000000     6.000000     7.000000     1.000000  10500.000000   
max    201903.000000    13.000000    14.000000     1.000000  10500.000000   

              mean      

In [49]:
data.describe()

,年月,count_0,count_1,is_deleted,price,mean,median,max,min,routine_flg,membership_period,period
count,3691.000000,3691.000000,3691.000000,3691.000000,3691.000000,3691.000000,3691.000000,3691.000000,3691.000000,3691.000000,3691.000000,3691.000000
mean,201838.083446,4.914115,5.303712,0.285018,8567.596857,5.333699,5.240585,8.028177,2.862368,0.869412,20.562178,16.107288
std,43.410314,2.271645,2.370925,0.451484,1969.784258,1.532915,1.659036,1.805222,1.709378,0.336995,13.748895,12.698223
min,201805.000000,1.000000,1.000000,0.000000,6000.000000,1.000000,1.000000,1.000000,1.000000,0.000000,2.000000,0.000000
25%,201808.000000,3.000000,4.000000,0.000000,6000.000000,4.333333,4.000000,7.000000,2.000000,1.000000,8.000000,5.000000
50%,201811.000000,5.000000,5.000000,0.000000,7500.000000,5.000000,5.000000,8.000000,2.000000,1.000000,19.000000,13.000000
75%,201901.000000,6.000000,7.000000,1.000000,10500.000000,6.166667,6.000000,9.000000,4.000000,1.000000,32.000000,26.000000
max,201903.000000,13.000000,14.000000,1.000000,10500.000000,10.666667,11.000000,14.000000,10.000000,1.000000,47.000000,46.000000


### ノック18：データ分析を深堀してみよう

In [50]:
print(agent.invoke("いくつかの観点で集計して、分かることを10個あげてください。必ず日本語でお願いします。"))



> Entering new AgentExecutor chain...
Thought: I need to analyze the dataframe and identify 10 insights. Since the dataframe contains information about customers, campaigns, and dates, I can look for patterns and trends. Here are some potential insights:

1.  **Time Trends:** Analyze how customer counts change over the years (2017, 2018, 2019) to see if there are seasonal trends or overall growth.
2.  **Campaign Popularity:** Identify which campaigns have the most customers.
3.  **Customer Segmentation by Campaign:**  Determine which campaigns are most popular with specific customer segments (e.g., C01, C03).
4.  **Customer Activity by Date:**  See if there are patterns in customer activity based on the start and end dates of campaigns.
5.  **Customer Demographics (Campaign ID):** Examine if certain campaign IDs correlate with specific customer behaviors.
6.  **Customer Activity by Customer ID:** Check for repeat customers and the frequency of their activity.
7.  **Impact of Campaign

### ノック19：Gradioで簡単なアプリケーションをつくろう

In [51]:
!pip install gradio==5.31.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 901.4 kB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.13.0
    Uninstalling gradio_client-1.13.0:
      Successfully uninstalled gradio_client-1.13.0
  Attempting uninstall: gradio
    Found existing installation: gradio 5.46.0
    Uninstalling gradio-5.46.0:
      Successfully uninstalled gradio-5.46.0


In [55]:
import gradio as gr
from PIL import Image

def preview_image(image: Image.Image) -> Image.Image:
    """
    アップロードされた画像をそのまま返して表示する関数
    """
    return image

iface = gr.Interface(
    fn=preview_image,
    inputs=gr.Image(type="pil", label="画像をアップロード"),
    outputs=gr.Image(type="pil", label="アップロード画像プレビュー"),
    title="画像プレビューツール",
    description="画像をアップロードすると、そのまま表示します。"
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ccf6f1beb6fd2d4349.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### ノック20：マルチモーダルLLMと組み合わせたアプリに拡張しよう

In [56]:
def describe_image(image_path):
    response = ollama.chat(
        model="gemma3",
        messages=[{
            "role": "user",
            "content": "この画像に写っているものを説明してください。",
            "images": [image_path]
        }]
    )
    return response["message"]["content"]

iface = gr.Interface(
    fn=describe_image,
    inputs=gr.Image(type="filepath", label="画像をアップロード"),
    outputs=gr.Textbox(label="画像説明"),
    title="マルチモーダル画像説明ツール",
    description="アップロードした画像を Gemma3 Vision で解析し、内容を日本語で説明します。"
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c65b671b56544395a8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
